
# Fiddler Chatbot Database Administration Notebook

This notebook provides administrative tools for managing the Fiddler chatbot database systems.
- **Vector Store Table**: `fiddler_doc_snippets_openai`
- **Chatbot Ledger Table**: `fiddler_chatbot_ledger` 
- **Keyspace**: `fiddlerai`
- **Current LLM Model**: `gpt-4-turbo`
- **Embedding Model**: `text-embedding-3-large`

### 1. **Data Inspection**
   - Views contents of vector store and ledger tables
   - Checks table structures and row counts
   - Analyzes data quality and completeness

### 2. **Data Analysis**
   - Examines chatbot interaction patterns
   - Reviews user feedback and ratings
   - Analyzes token usage and costs

### 3. **Data Export**
   - Exports data for Fiddler AI monitoring
   - Creates CSV files for analysis
   - Prepares baseline and event datasets

### 4. **Maintenance Operations**
   - Data cleanup and validation
   - Performance monitoring
   - Schema validation

**⚠️ Important**: This notebook has been updated to align with the current system configuration as of 2025. Legacy migration code has been preserved but marked as historical reference.

In [ ]:
# Import required libraries
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import os
import pandas as pd
from datetime import datetime

print( os.getcwd() )
os.chdir('../')
print( os.getcwd() )

from config import CONFIG_CHATBOT_OLD as config  # noqa: E402

print(f"Cassandra version: {cassandra.__version__}")
print("Current system configuration loaded:")
print(f"  - Keyspace:        {config['ASTRA_DB_KEYSPACE']}")
print(f"  - Vector table:    {config['ASTRA_DB_TABLE_NAME']}")
print(f"  - Ledger table:    {config['ASTRA_DB_LEDGER_TABLE_NAME']}")
print(f"  - LLM Model:       {config['OPENAI_LLM_MODEL']}")
print(f"  - Embedding Model: {config['OPENAI_EMBEDDING_MODEL']}")

In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB, if yours is different update the file name below
# Database connection setup using current configuration
print("🔄 Setting up database connection...")

cloud_config = {
    'secure_connect_bundle': '../'+config["ASTRA_DB_SECURE_BUNDLE_PATH"]
}

# Get authentication token from environment
ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
if not ASTRA_DB_APPLICATION_TOKEN:
    raise ValueError("ASTRA_DB_APPLICATION_TOKEN environment variable is required")

# Create connection
auth_provider = PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Set keyspace using configuration
keyspace = config["ASTRA_DB_KEYSPACE"]
session.set_keyspace(keyspace)

print("✅ Connected to DataStax Astra DB using keyspace: {keyspace}")

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
# session.default_fetch_size = None

In [ ]:
#rid = "<class 'uuid.UUID'>"
# rows = session.execute("DELETE row_id FROM fiddler_chatbot_ledger WHERE model_name= '' ")
# df = rows._current_rows
# df

In [ ]:
# Query the main vector store table
vector_table = config["ASTRA_DB_TABLE_NAME"]
print(f"📊 Querying vector store table: {vector_table}")

query = f'SELECT * FROM {vector_table}'
rows = session.execute(query)
df_docs = rows._current_rows

print(f"📈 Retrieved {len(df_docs)} documents from vector store")
print(f"📋 Table schema: {list(df_docs.columns)}")

# Display basic statistics
if len(df_docs) > 0:
    print(f"📏 Vector dimensions: {len(df_docs.iloc[0]['vector']) if 'vector' in df_docs.columns else 'N/A'}")
    print(f"📝 Sample document preview: {str(df_docs.iloc[0]['body_blob'])[:100]}..." if 'body_blob' in df_docs.columns else "")

df_docs

In [ ]:
# Query the current chatbot interaction ledger
ledger_table = config["ASTRA_DB_LEDGER_TABLE_NAME"]
print(f"📊 Querying chatbot ledger table: {ledger_table}")

query = f"SELECT * FROM {ledger_table} LIMIT 100"
rows = session.execute(query)
df_ledger = rows._current_rows

if len(df_ledger) > 0:
    df_ledger = df_ledger.sort_values(by=['ts'], ascending=False)
    print(f"📈 Retrieved {len(df_ledger)} chatbot interactions")
    print(f"📋 Current schema: {list(df_ledger.columns)}")
    
    # Check for any missing or extra fields
    actual_fields = set(df_ledger.columns)
            
    print(f"🚀 Latest interaction timestamp: {df_ledger.iloc[0]['ts'] if 'ts' in df_ledger.columns else 'N/A'}")
    print(f"🤖 Current model in use: {df_ledger.iloc[0]['model_name'] if 'model_name' in df_ledger.columns else 'N/A'}")
else:
    print("⚠️  No data found in ledger table")
    df_ledger = pd.DataFrame()

# Display the latest 5 interactions
df_ledger.head(5)


In [ ]:
# =============================================================================
# DATA EXPORT FOR FIDDLER AI MONITORING
# =============================================================================
# Export chatbot interaction data for analysis and monitoring

ledger_table = config["ASTRA_DB_LEDGER_TABLE_NAME"]
print(f"📊 Exporting FIRST 100 rows of data from {ledger_table}")

query = f'SELECT * FROM {ledger_table} LIMIT 100'
rows = session.execute(query)
df_export = rows._current_rows

if len(df_export) > 0:
    # Sort by timestamp
    df_export = df_export.sort_values(by=['ts'])
    
    # Create baseline dataset (first 50 interactions for model baseline)
    baseline_count = min(50, len(df_export))
    df_baseline = df_export.iloc[:baseline_count].copy()
    df_baseline = df_baseline.drop(columns=['ts'], errors='ignore')  # Remove timestamp for baseline
    
    # Create events dataset (remaining interactions for monitoring)
    df_events = df_export.iloc[baseline_count:].copy()
    
    # Export to CSV files with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    baseline_filename = f'chatbot_baseline_{timestamp}.csv'
    events_filename = f'chatbot_events_{timestamp}.csv'
    
    df_baseline.to_csv(baseline_filename, index=False)
    df_events.to_csv(events_filename, index=False)
    
    print(f"✅ Exported baseline data: {baseline_filename} ({len(df_baseline)} rows)")
    print(f"✅ Exported events data: {events_filename} ({len(df_events)} rows)")
    print(f"📊 Total interactions: {len(df_export)}")
    
    # Display export summary
    if len(df_export) > 0:
        print(f"📅 Date range: {df_export['ts'].min()} to {df_export['ts'].max()}")
        print(f"🤖 Models used: {df_export['model_name'].unique().tolist() if 'model_name' in df_export.columns else 'N/A'}")
else:
    print("⚠️  No data available for export")

In [ ]:
# =============================================================================
# MAINTENANCE AND CLEANUP OPERATIONS (USE WITH CAUTION)
# =============================================================================
# These operations can modify or delete data - use only when necessary
# All operations are commented out for safety

# WARNING: The following operations are DESTRUCTIVE and should only be used
# by administrators who understand the consequences

# Clean up specific problematic records:
# session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

# Remove legacy tables (ONLY if migration is complete and verified):
# session.execute("DROP TABLE fiddler_chatbot_history")

# Clear vector store (ONLY for complete rebuild):
# vector_table = config["ASTRA_DB_TABLE_NAME"]
# session.execute(f"TRUNCATE TABLE {vector_table}")

# Clear chatbot history (ONLY for fresh start):
# ledger_table = config["ASTRA_DB_LEDGER_TABLE_NAME"]
# session.execute(f"TRUNCATE TABLE {ledger_table}")

# Current system status check
print("📋 Current system status:")
print(f"   - Vector table:  {config['ASTRA_DB_TABLE_NAME']}")
print(f"   - Ledger table:  {config['ASTRA_DB_LEDGER_TABLE_NAME']}")
print(f"   - Current model: {config['OPENAI_LLM_MODEL']}")
print("🔐 All maintenance operations are commented out for safety")
print("⚠️  Uncomment and execute maintenance operations only if you understand the consequences")

In [ ]:
# =============================================================================
# UTILITY FUNCTIONS FOR CURRENT SYSTEM ANALYSIS
# =============================================================================
# Helper functions for analyzing the current chatbot system

def get_system_health_summary():
    """Get a comprehensive health summary of the current system"""
    print("🏥 SYSTEM HEALTH SUMMARY")
    print("=" * 50)
    
    # Check vector store
    vector_table = config["ASTRA_DB_TABLE_NAME"]
    rows = session.execute(f'SELECT COUNT(*) as count FROM {vector_table}')
    vector_count : int= len( list(rows)[0] )
    print(f"📚 Vector Store ({vector_table}): {vector_count} documents")
    
    # Check chatbot ledger
    ledger_table = config["ASTRA_DB_LEDGER_TABLE_NAME"]
    rows = session.execute(f'SELECT COUNT(*) as count FROM {ledger_table}')
    ledger_count : int = len( list(rows)[0] )
    print(f"💬 Chatbot Interactions ({ledger_table}): {ledger_count} records")
    
    if ledger_count > 0:
        # Get recent activity - fetch all data and sort in pandas since Cassandra can't ORDER BY non-clustering columns
        recent_query = f'SELECT model_name, ts FROM {ledger_table}'
        rows = session.execute(recent_query)
        df_recent = rows._current_rows
        
        if len(df_recent) > 0:
            # Sort by timestamp in pandas and get the latest
            df_recent = df_recent.sort_values(by=['ts'], ascending=False)
            latest_row = df_recent.iloc[0]
            latest_ts = latest_row['ts']
            latest_model = latest_row['model_name']
            print(f"🕐 Latest interaction: {latest_ts}")
            print(f"🤖 Latest model used: {latest_model}")
    
    print(f"⚙️  Configured model: {config['OPENAI_LLM_MODEL']}")
    print(f"🧠 Embedding model: {config['OPENAI_EMBEDDING_MODEL']}")
    print("=" * 50)

def get_recent_interactions(limit=5):
    """Get the most recent chatbot interactions"""
    ledger_table = config["ASTRA_DB_LEDGER_TABLE_NAME"]
    query = f'SELECT row_id, prompt, response, model_name, ts FROM {ledger_table}'
    
    try:
        rows = session.execute(query)
        df_recent = rows._current_rows
        
        if len(df_recent) > 0:
            # Sort by timestamp in pandas and get the most recent records
            df_recent = df_recent.sort_values(by=['ts'], ascending=False)
            df_recent = df_recent.head(limit)
            
            print(f"💬 RECENT INTERACTIONS (Last {len(df_recent)})")
            print("=" * 60)
            for i, (_, row) in enumerate(df_recent.iterrows(), 1):
                print(f"{i}. {row['ts']} - {row['model_name']}")
                prompt = str(row['prompt']) if row['prompt'] is not None else "No prompt"
                response = str(row['response']) if row['response'] is not None else "No response"
                print(f"   Q: {prompt[:100]}..." if len(prompt) > 100 else f"   Q: {prompt}")
                print(f"   A: {response[:100]}..." if len(response) > 100 else f"   A: {response}")
                print()
            print("=" * 60)
            
            return df_recent.to_dict('records')
        else:
            print("💬 No interactions found")
            return []
        
    except Exception as e:
        print(f"⚠️  Could not retrieve recent interactions: {e}")
        return []

# Run health summary by default
get_system_health_summary()
get_recent_interactions(10)


In [9]:
from langchain_community.vectorstores import Cassandra as CassandraVectorStore
from langchain_openai import OpenAIEmbeddings
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import time

# =============================================================================
# VECTOR SIMILARITY SEARCH UTILITY
# =============================================================================
# This cell replicates the vector search functionality from the main chatbot
# to allow debugging and testing of retrieval queries

"""
🚀 Vector Search Utility Ready!
📝 Example usage:
    documents = test_vector_search('What is Fiddler?')
    documents = test_vector_search('How to monitor models?', k=5)
    documents = test_vector_search('API documentation', k=3, show_details=False)
"""

def test_vector_search(query: str, k: int = 3, show_details: bool = True):
    """
    Perform vector similarity search using the same logic as the main chatbot.
    
    Args:
        query (str): The search query (e.g., "What is Fiddler?")
        k (int): Number of documents to retrieve (default: 3, same as chatbot)
        show_details (bool): Whether to show detailed document content
    
    Returns:
        List of retrieved documents
    """
    print(f"🔍 Performing vector search for: '{query}'")
    print(f"📊 Retrieving top {k} most relevant documents...")
    print("=" * 60)
    
    # Set up embeddings using the same configuration as chatbot
    embeddings = OpenAIEmbeddings(
        model=config["OPENAI_EMBEDDING_MODEL"], 
        dimensions=1536
    )
    
    # Create a new session specifically for vector search (without pandas_factory)
    # The existing session has pandas_factory which conflicts with LangChain's expectations
    cloud_config = {
        'secure_connect_bundle': '../'+config["ASTRA_DB_SECURE_BUNDLE_PATH"]
    }
    
    ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
    if not ASTRA_DB_APPLICATION_TOKEN:
        raise ValueError("ASTRA_DB_APPLICATION_TOKEN environment variable is required")
    
    # Create a separate connection for vector operations (without custom row factory)
    auth_provider = PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
    vector_cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    vector_session = vector_cluster.connect()
    vector_session.set_keyspace(config["ASTRA_DB_KEYSPACE"])
    
    # Create Cassandra vector store using the dedicated session
    vector_store = CassandraVectorStore(
        embedding=embeddings,
        session=vector_session,
        keyspace=config["ASTRA_DB_KEYSPACE"],
        table_name=config["ASTRA_DB_TABLE_NAME"]
    )
    
    # Perform similarity search with timing
    start_time = time.time()
    documents = vector_store.similarity_search(query, k=k)
    search_time = time.time() - start_time
    
    print(f"⏱️  Search completed in {search_time:.3f} seconds")
    print(f"📄 Retrieved {len(documents)} documents")
    print("=" * 60)
    
    if not documents:
        print("⚠️  No documents found for the query")
        return []
    
    # Display results
    for i, doc in enumerate(documents, 1):
        print(f"\n📋 Document {i}:")
        print(f"   Content length: {len(doc.page_content)} characters")
        
        if show_details:
            # Show first 200 characters of content
            content_preview = doc.page_content #.replace('\n', ' ')
            print(f"   Preview: {content_preview}...")
            
            # Show metadata if available
            if hasattr(doc, 'metadata') and doc.metadata:
                print(f"   Metadata: {doc.metadata}")
        else:
            # Just show first 100 characters
            content_preview = doc.page_content #.replace('\n', ' ')
            print(f"   Preview: {content_preview}...")
    
    print("\n" + "=" * 60)

    vector_cluster.shutdown()
    return documents


test_documents = test_vector_search("tell me about all the releases after 25.10?", k=5)


🔍 Performing vector search for: 'tell me about all the releases after 25.10?'
📊 Retrieving top 5 most relevant documents...
⏱️  Search completed in 2.213 seconds
📄 Retrieved 5 documents

📋 Document 1:
   Content length: 2943 characters
   Preview: [CONTEXT: Header 1: Release Notes]

#### What's New and Improved:  
* Performance Analytics (Preview) Embedded in Monitoring Charts
* Visualize performance analytics charts as part of the root cause analysis flow for Binary Classification, Multiclass Classification, and Regression models, spanning from confusion matrices, precision recall charts, prediction scatterplots and more.  
### Release 24.10 Notes  
#### Client Version  
Client version 3.3+ is required for the updates and features mentioned in this release.  
#### What's New and Improved:  
* Support for applied segments in monitoring charts
* Create and apply segments dynamically in monitoring charts for exploratory analysis without requiring them to be saved to the model.
* User-Def